<a href="https://colab.research.google.com/github/aysegulbulbul/Latent-Dirichlet-Allocation-LDA-/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Başka kütüphaneler de eğmodelde gerekli olucak. Onlar da alltaki kodlarda yer alıyor.

In [ ]:
!pip install opendatasets

In [ ]:
#Kaggle'dan bu şekilde locale indirmeden veri setini alabilirsiniz.
#Kullanıcı adınızı ve token şifrenizi isteyecektir. Ayarlar kısmından yeni token oluşturarak alabilirsiniz
#İstediğiniz datasetin adresini yazarak indirebilirsiniz.

import opendatasets as od

od.download("https://www.kaggle.com/datasets/suleymancan/turkishnews70000/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aysegulbulbul
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/suleymancan/turkishnews70000


100%|██████████| 62.6M/62.6M [00:03<00:00, 18.9MB/s]


In [ ]:
df= pd.read_csv("/content/turkishnews70000/turkish_news_70000.csv",index_col="id")

df.head(5)

,main_image,published,site,text,title,url
id,,,,,,
72337,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:33:00.000+02:00""","""diken.com.tr""","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9...","""http://www.diken.com.tr/yatirim-bankasi-dolar..."
72338,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T13:21:00.000+02:00""","""diken.com.tr""","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""PİAR Araştırma: Adana ve Antalya’da ‘millet i...","""http://www.diken.com.tr/piar-arastirma-adana-..."
72339,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T12:20:00.000+02:00""","""diken.com.tr""","""Renaissance Capital: Merkez Bankası bu hafta ...","""Renaissance Capital: Merkez Bankası bu hafta ...","""http://www.diken.com.tr/renaissance-capital-m..."
72340,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T11:53:00.000+02:00""","""diken.com.tr""","""Ağrı’daki İYİ Partililerin istifa gerekçesi: ...","""Ağrı’daki İYİ Partililerin istifa gerekçesi: ...","""http://www.diken.com.tr/agridaki-iyi-partilil..."
72341,"""http://www.diken.com.tr/wp-content/uploads/20...","""2019-03-04T11:52:00.000+02:00""","""diken.com.tr""","""Otomobil pazarı iki ayda yarı yarıya eridi 04...","""Otomobil pazarı iki ayda yarı yarıya eridi""","""http://www.diken.com.tr/otomobil-pazari-iki-a..."


In [ ]:
df['text'] = df['text'].astype(str)

In [ ]:
#Konu modellemesi örneğimiz için LDA modelinde text sütununu kullanmamız yeterli olucaktır.

haber_dataset = df[["text"]]
haber_dataset.head(5)

,text
id,
72337,"""Yatırım bankası: Dolar/TL üçüncü çeyrekte 8.9..."
72338,"""PİAR Araştırma: Adana ve Antalya’da ‘millet i..."
72339,"""Renaissance Capital: Merkez Bankası bu hafta ..."
72340,"""Ağrı’daki İYİ Partililerin istifa gerekçesi: ..."
72341,"""Otomobil pazarı iki ayda yarı yarıya eridi 04..."


#Metin Temizleme ve Metin Normalizasyonu

* Tüm harfleri küçük duruma getiriyoruz
* Noktalama işaretlerini kaldırmalıyız
* Etkisiz kelimeleri (stopwords) kaldırıcaz (için,ama,ve gibi)

In [ ]:
#Veri temizleme işlemi için bir fonk.tanımlıycaz ve bu fonksiyonu tüm verilerde kullanıcaz
#Öncesinde noktalama işaretlerini ve stopword kelimeleri tanımlayacağız.

In [ ]:
#Gerekli kütüphaneleri yüklüyorum

import re #(regular expression)
import string
import nltk
from nltk.corpus import stopwords


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nok_isaretleri = string.punctuation
etkisiz_kelimeler= stopwords.words('turkish') #nltk'den etkisi olmayan türkçe kelimeleri seçiyor.Fakat biz de ekleyebiliriz.
etkisiz_kelimeler.extend(["bir","kadar","sonra"])

print(nok_isaretleri)
print(etkisiz_kelimeler)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'bir', 'kadar', 'sonra']


In [ ]:
def veri_temizleme(metin):
  metin=metin.lower() #Tüm harfler küçük harfe dönüştürülür.
  metin=metin.replace("\\n", " ") #Veri setindeki yeni satırlar boşluk karakteri ile yer değiştiriyoruz.

  #Kesme işareti ve sonrasındaki karakterlerin kaldırılması
  metin= re.sub("'(\W+)", "",metin)  # \W = boşluklar, noktalama işaretleri ve diğer özel karakterler eşleştirir.
  metin= re.sub("[“,”]", "",metin)

  metin= re.sub("[0-9]+", "",metin)  #Sayıların kaldırılması
  #Noktalama işaretlerinin kaldırılması
  metin= "".join(list(map(lambda x:x if x not in nok_isaretleri else " ",metin)))
  #Etkisiz elemanların kaldırılması
  metin= " ".join([i for i in metin.split() if i not in etkisiz_kelimeler])
  #Metinde tek kalan harfleri çıkartma
  metin= " ".join([i for i in metin.split() if len(i) > 1])

  return metin

In [ ]:
# Burada text sütunundaki normal metinlerin yerine temizlenmiş veriyi uyguluyruz ve artık text sütunumuz temizlemiş oluyor.

haber_dataset['text'] = haber_dataset['text'].apply(lambda x: veri_temizleme(x))

<ipython-input-12-595fc33ea9cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haber_dataset['text'] = haber_dataset['text'].apply(lambda x: veri_temizleme(x))


In [ ]:
haber_dataset['temiz_veri'] = haber_dataset.iloc[6].text

<ipython-input-13-458f5bf5875f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haber_dataset['temiz_veri'] = haber_dataset.iloc[6].text


In [ ]:
haber_dataset['temiz_veri'].iloc[6]

'ysk başkanı güven den seçim güvenliğiaçıklaması şubat cumartesi son güncelleme şubat cumartesi ysk başkanı sadi güven vatandaşlarımızın kendi oy kullandığı sandığın tutanağını sitemizden görme imkanları var vatandaşlarımız bunları kontrol etsin dedi yüksek seçim kurulu ysk başkanı sadi güven vatandaşların oy kullandıkları sandıktan çıkan sonucun ıslak imzalı tutanağını sandığın ilçe il birleştirmesini türkiye geneline ysk sitesinden kontrol edebileceğini belirterek vatandaşlarımız bunları kontrol etsin herkes kolay erişebilsin sistem basitleşti şimdi kolay görülüyor herkes kendi oy kullandığı sandıktan çıkan sonuca baksa spekülasyonlar ortadan kalkar dedi güven yaptığı açıklamada seçim süreciyle ilgili bilgi verdi seçim iş işlemlerini yürütmek üzere şubat tarihli milletvekilleri seçimi kanunu kurulan ysk nin bugün yaşını kutladığını anımsatan güven kurumun yıldır seçimlerin düzen içinde yapılması amacıyla çalıştığını kaydetti sadi güven mart ta yapılacak mahalli idareler seçimlerinin 

In [ ]:
# Verideki 'cleaned_text' sütunundaki her bir metni tokenize ederek 'cleaned_text_token' sütunu oluşturur.

haber_dataset['temiz_veri_token'] = haber_dataset['temiz_veri'].apply(lambda x: x.split())

<ipython-input-15-01f7cf0704cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haber_dataset['temiz_veri_token'] = haber_dataset['temiz_veri'].apply(lambda x: x.split())


In [ ]:
#haber_dataset.drop(['text'], axis=1, inplace=True)

In [ ]:
haber_dataset.head(5)

,text,temiz_veri,temiz_veri_token
id,,,
72337,yatırım bankası dolar tl üçüncü çeyrekte ’ı gö...,ysk başkanı güven den seçim güvenliğiaçıklamas...,"[ysk, başkanı, güven, den, seçim, güvenliğiaçı..."
72338,pi̇ar araştırma adana antalya’da ‘millet ittif...,ysk başkanı güven den seçim güvenliğiaçıklamas...,"[ysk, başkanı, güven, den, seçim, güvenliğiaçı..."
72339,renaissance capital merkez bankası hafta baz p...,ysk başkanı güven den seçim güvenliğiaçıklamas...,"[ysk, başkanı, güven, den, seçim, güvenliğiaçı..."
72340,ağrı’daki i̇yi̇ partililerin istifa gerekçesi ...,ysk başkanı güven den seçim güvenliğiaçıklamas...,"[ysk, başkanı, güven, den, seçim, güvenliğiaçı..."
72341,otomobil pazarı iki ayda yarı yarıya eridi oto...,ysk başkanı güven den seçim güvenliğiaçıklamas...,"[ysk, başkanı, güven, den, seçim, güvenliğiaçı..."


#LDA MODEL TRAINING
Bu kodda, LDA modeli oluşturulup eğitilmiş ve metinlerdeki en belirgin konular çıkarılır

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.7 MB/s eta 0:00:00


In [ ]:
import gensim
import pyLDAvis.gensim # for visualization
from gensim.models import CoherenceModel

In [ ]:
# Tokenize edilmiş metinlerden oluşan bir liste oluşturur.
tokenize_metin = haber_dataset['temiz_veri_token']

# Tokenize edilmiş metinlerden bir kelime sözlüğü (dictionary) oluşturur.
kelime_list = gensim.corpora.Dictionary(tokenize_metin)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Belge terim matrisi oluşturur (her belge için kelimelerin sıklığını içeren bir liste).
document_term_matrix = [kelime_list.doc2bow(term) for term in tokenize_metin]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# LDA modelini oluşturur ve eğitir.

lda_model = gensim.models.ldamodel.LdaModel(
                                            corpus= document_term_matrix,
                                            id2word= kelime_list,
                                            num_topics = 20,
                                            passes = 10
                                           )

'''
corpus=document_term_matrix: LDA modeline verilecek belge terim matrisini belirtir.
id2word=word_list: LDA modeline kelime sözlüğünü (dictionary) belirtir.
num_topics=15: Modelin bulmasını istediğiniz gizli konu sayısını belirtir.
passes=10: Modelin tüm veri seti üzerinde kaç kez geçeceğini belirtir.
'''

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'\ncorpus=document_term_matrix: LDA modeline verilecek belge terim matrisini belirtir.\nid2word=word_list: LDA modeline kelime sözlüğünü (dictionary) belirtir.\nnum_topics=15: Modelin bulmasını istediğiniz gizli konu sayısını belirtir.\npasses=10: Modelin tüm veri seti üzerinde kaç kez geçeceğini belirtir.\n'

In [ ]:
# En sık tekrarlanan kelimeleri içeren konuları yazdırır.

topics = lda_model.print_topics(num_words=9)

for topic in topics:
    print(topic)

(0, '0.040*"güven" + 0.028*"seçim" + 0.021*"ysk" + 0.017*"oy" + 0.016*"bin" + 0.013*"olduğunu" + 0.012*"milyon" + 0.010*"kolay" + 0.009*"kullandığı"')
(1, '0.037*"güven" + 0.028*"seçim" + 0.022*"ysk" + 0.020*"bin" + 0.019*"oy" + 0.014*"milyon" + 0.013*"olduğunu" + 0.012*"kolay" + 0.009*"ıslak"')
(2, '0.037*"güven" + 0.024*"seçim" + 0.024*"ysk" + 0.015*"oy" + 0.013*"bin" + 0.012*"kolay" + 0.012*"milyon" + 0.012*"olduğunu" + 0.009*"sadi"')
(3, '0.036*"güven" + 0.021*"seçim" + 0.020*"ysk" + 0.019*"oy" + 0.015*"bin" + 0.014*"olduğunu" + 0.012*"milyon" + 0.010*"siyasi" + 0.010*"kolay"')
(4, '0.035*"güven" + 0.027*"seçim" + 0.024*"ysk" + 0.019*"bin" + 0.016*"oy" + 0.013*"milyon" + 0.012*"olduğunu" + 0.012*"kolay" + 0.010*"herkes"')
(5, '0.032*"güven" + 0.030*"seçim" + 0.029*"ysk" + 0.019*"oy" + 0.018*"bin" + 0.013*"milyon" + 0.012*"olduğunu" + 0.011*"kolay" + 0.010*"sadi"')
(6, '0.029*"güven" + 0.025*"seçim" + 0.024*"ysk" + 0.015*"bin" + 0.015*"oy" + 0.014*"milyon" + 0.012*"olduğunu" + 0.011

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#MODEL VISUALIZATION

In [ ]:
# pyLDAvis kullanarak LDA modelinin görselleştirilmesi için gerekli ortamı sağlar.
pyLDAvis.enable_notebook()

# LDA modelinin görselleştirilmesi için bir nesne hazırlar.
gorsel = pyLDAvis.gensim.prepare(lda_model, document_term_matrix, kelime_list, mds='mmds')

# Görselleştirmeyi görüntüler.
pyLDAvis.display(gorsel)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


#Coherence Score
LDA (Latent Dirichlet Allocation) modeli kullanılarak bir dizi farklı konu sayısı (topic number) için tutarlılık (coherence) skoru hesaplanıyor.

In [ ]:
topic_number_range_list = range(9,30,3)

coherence_score_list = list()
topic_number_list = list()

for topic_number in topic_number_range_list:
    lda_model = gensim.models.ldamodel.LdaModel(
                                                corpus= document_term_matrix,
                                                id2word= kelime_list,
                                                num_topics = topic_number,
                                                passes = 10
                                                )

    coherence_model_lda = CoherenceModel(
                                        model = lda_model,
                                        texts = tokenize_metin,
                                        dictionary = kelime_list,
                                        coherence='c_v'
                                        )

    temp_coherence_score_lda = coherence_model_lda.get_coherence()
    coherence_score_list.append(temp_coherence_score_lda)
    topic_number_list.append(topic_number)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: 

 LDA modeli ile oluşturulan konuların kelimelerle birlikte yazdırılması ve ardından bu konuların etkileşimli bir görselleştirme aracı olan pyLDAvis ile görselleştirilmesi

In [ ]:
plt.plot(topic_number_list, coherence_score_list, "-")
plt.xlabel("Topic Numbers")
plt.ylabel("Coherence Scores")

plt.show()

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, document_term_matrix, dictionary=lda_model.id2word, mds='mmds')
pyLDAvis.display(vis)